**Задача**\
Для отчета нужно расчитывать план, исходя из количества будних дней месяца без учета понедельников. 
Отчет работает на сервере, на котором нет прав на создание предварительных витрин - нужно уложиться в одно подключение к экселю.

**Решили реализовать вариант решения** через case - when.
Для каждого месяца количество рабочих дней (без понедельника) будет зависеть от двух факторов:
- сколько дней в месяце (28/29/30/31),
- на какой день недели пришлось первое число месяца.
Т.о. должны получить 28 вариантов. 

Для реализации этого варианта **напишем две функции**:
- первая будет собирать таблицу, в которой посчитано количество календарных дней для каждого дня недели по каждому варианту месяца
- вторая собирать текста скрипта, который можно будет подставить в запрос.


Первая функция получилась довольно кондовой, но рабочей.\
Возможно, получится более изящный вариант подсчета, если сделать алгоритм, если сначала создать базовый датафрейм исходя из предпосылок:
- Минимальная длина месяца - 28 дней. Дней недели - 7. Значит минимальное число календарных дней каждого дня недели - 4.
- Вопрос - как распределятся оставшиеся дни для месяцев, когда дней >28. 
- Они будут распределяться последовательно по дням недели, начиная с того, на который пришлось первое число месяца.
Т.о. можно создать базовый датафрейм, где у всех вариантов по 4 календарных дня на каждый день недели. А далее циклом по остатку от деления количества дней в месяце на 7 прибавить по 1 дням недели, начиная с того, на который пришлось начало месяца.

In [3]:
import pandas as pd

Расчитаем количество дней недели для каждого месяца

In [135]:
def week_days_in_month ():
    """
    Функция собирает датафрейм, в котором подсчитывается количество дней недели в месяце
    в зависимости от того, сколько календарных дней в месяце и в какой день недели он начался.
    Поля:
    m_days_cnt - кол-во календарных дней в месяце
    w_day_start - день недели, в который начался месяц
    w_day - номер дня недели, по котором считаем кол-во вхождений в месяц
    days_cnt - воличество календарных дней с таким днем недели в таком месяце

    Все счетчики идут от 1, чтобы было интуитивно понятней, о каком дне недели речь.
    Кстати, началом недели (первым днем) считаем понедельник.
    """
    # Варианты количества дней в месяце
    m_days_cnt = [28, 29, 30, 31]
    # Дни недели, когда может начаться месяц, где 1 - понедельник
    w_day_start = [1, 2, 3, 4, 5, 6, 7]    
    # Для каждого варианта кол-ва дней в месяце
    md_i = 1 # счетчик вариантов месяца
    for md in m_days_cnt:
        # Для каждого варианта дня начала месяца 
        wds_i = 1 # счетчик дней начала
        for wds in w_day_start:                   
            # Для каждого дня месяца в таком варианте
            week_day_num = wds
            week_num = 1
            cycle_counter = 1
            for d in range(1,md+1):
                # Добавляем 1 к кол-ву дней
                if week_num == 1:
                    wds_frame = pd.DataFrame({'m_days_cnt': [md],
                                              'w_day_start': [wds], 
                                              'w_day': [week_day_num],
                                              'days_cnt': 1})
                else: 
                    d_cnt = (wds_frame_final
                             .query('(m_days_cnt == @md) & (w_day_start == @wds) & (w_day == @week_day_num)')['days_cnt']
                             .values[0] + 1)
                    wds_frame = pd.DataFrame({'m_days_cnt': [md],
                                              'w_day_start': [wds], 
                                              'w_day': [week_day_num],
                                              'days_cnt': [d_cnt]})
                # Если это первый проход цикла, то создаем результирующит датафрейм, если нет, то добавляем к результирующему 
                if d == 1:
                    wds_frame_final = wds_frame
                elif d != 1 and week_num ==1:
                    wds_frame_final = pd.concat([wds_frame_final, wds_frame], axis = 0)
                elif d != 1 and week_num !=1:
                    #print("md: ", md, "wds: ",wds, " d: ", d, " cycle_counter: ",cycle_counter, " week_day_num: ", week_day_num, " week_num: ", week_num, " d_cnt: ", d_cnt)
                    wds_frame_final.loc[(wds_frame_final['m_days_cnt']==md)&(wds_frame_final['w_day_start']==wds)&(wds_frame_final['w_day']==week_day_num)&(wds_frame_final['days_cnt']==d_cnt-1)] = wds_frame
                
                # Переставляем счетчик дня недели, чтобы он возвращался к понедельнику после воскресенья
                if week_day_num < 7:
                    week_day_num += 1
                else:
                    week_day_num = 1
                
                if cycle_counter == 7:
                    week_num += 1
                
                if cycle_counter < 7:
                    cycle_counter += 1
                else:
                    cycle_counter = 1
                
            # Если первый вариант начала месяца, он начинает датафрейм для типа кол-ва дней месяца
            if wds_i == 1:
                md_frame = wds_frame_final
            else:
                md_frame = pd.concat([md_frame, wds_frame_final], axis = 0)
            # Переставляем счетчик
            wds_i +=1
        # Если первый вариант начала кол-ва дней в месяце, он начинает результирующий датафрейм 
        if md_i == 1:
            final_frame = md_frame
        else:
            final_frame = pd.concat([final_frame, md_frame], axis = 0)
        # Переставляем счетчик
        md_i +=1
    return final_frame

In [136]:
final_frame = week_days_in_month ()

In [138]:
final_frame.query('m_days_cnt == 31').head(7)

,m_days_cnt,w_day_start,w_day,days_cnt
0,31,1,1,5
0,31,1,2,5
0,31,1,3,5
0,31,1,4,4
0,31,1,5,4
0,31,1,6,4
0,31,1,7,4


Напишем функцию, которая будет из результирующего датафрейма сформирует sql-код с расчетом количества дней.

In [151]:
def script_writer (week_days):
    """
    Функция принимает на вход датафрейм с количеством дней по каждому дню недели
    и возвращает текст скрипта, который можно будет добавить в case запроса в t-sql.
    """
    pivot = (week_days
             .query('w_day.between(2,5)') # берем дни со вт по пт
             .pivot_table(index=['m_days_cnt','w_day_start'], values = 'days_cnt', aggfunc = 'sum')
             .reset_index())
    for index, row in pivot.iterrows():
        print('when day(eomonth(getdate())) =', row['m_days_cnt'], 
              "and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) =", row['w_day_start'],  # сравниваем со последним днем предыдущего месяца, т.к. t-sql считает дни недели с воскресенья
              "then ", row['days_cnt'])

In [152]:
script_writer(final_frame)

when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 1 then  16
when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 2 then  16
when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 3 then  16
when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 4 then  16
when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 5 then  16
when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 6 then  16
when day(eomonth(getdate())) = 28 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 7 then  16
when day(eomonth(getdate())) = 29 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 1 then  16
when day(eomonth(getdate())) = 29 and DATEPART(weekday,eomonth(dateadd(month,-1,getdate()))) = 2 then  17
when day(eomonth(getdate())) = 29 and DATEPART